# 1. Problem definition
This will be an extended version of what you submitted for your proposal. It should detail the exact nature of the problem you are trying to solve, along with why this problem is interesting/significant, and why AI approaches are a good fit. [HC: #rightproblem].

In this assignment, I implemented a general n*n tic tac toe game with an AI opponent, using minimax algorithm. This problem is very interesting as the tree size of possible game states grow exponentially as the tic tac toe game gets bigger.

* The tic tac toe game grid can be viewed as a tree, where each node is a distinct game status. When users choose a move, they effectively move to a game node on the following layer. A player wins when they manage to occupy one of the winning states with their designated symbol. The winning states are where a user fully occupies a matrix row, column, principle diagonal or minor diagonal.
* As such, as the user and AI approach the n*n layer, they are faced with one of the following scenarios:  
    1. User win: if the current node containts one of their winning states.
    2. AI win: if the current node containts one of their winning states.
    3. Draw: if the current node doesn't contain any win state for any player and the tree reaches its end (no cells left)

# 2. Solution Specification
This section should describe your approach to solving the problem described in the problem definition section. It should detail the steps taken to solve the problem, including the AI method or methods that you have adopted, and how you applied those methods to produce your solution. [HCs: #breakitdown, #algorithms, LOs: will depend on those you nominated in your proposal].

* In order to effectively tackle this problem, I made sure to breakdown the software design and implementation process of the tic-tac-toe game to the most tractable components. Additionally, I assigned a time limit for each task, in order to ensure that I manage to complete all tasks. Lastly, I further broke down tasks multiple times while working on them to achieve more tractable components. My breakdown of the problem is as follows:
    1. Game state calculation
        First, I worked on the Boards class, defining how the game will be represented in code, using relevant methods and attributes. Then, in order to ensure that the game works with any dimension n*n, I designed and implemented the MathOps class that provide helper function for win states calculations.
    1. Game mechanics:
        Then, I designed the game in two user players mode in order in order to thorouhgly test the game mechanics and the user input without introducing the AI player yet.
    3. AI player:
        Lastly, I designed the minimax_optimal (Heineman, 2016) and game_status functions to implement the ai_turn function. Afterwards, I tested the algorithm many times to evaluate its efficacy and sport any bugs. 

# 3. Analysis of Solution
This section should present an analysis of your proposed solution operating on some relevant test cases. It should describe the test cases used, and relevant results using appropriate representations (e.g. tables and figures). [HCs: #simulation, #professionalism. LOs: will depend on those you nominated in your proposal]

Overall, I tested my solutions with varying user inputs and the AI managed to win many times. Overall, the AI's performance is very good. However, as the board size grows beyond 4*4, it's important to introduce alpha beta pruning as that will reduce the number of nodes that minimax searches at each ai turn. This will lead to a higher AI move calculation speed and probability of winning the game. The solution along with testcases are provided in the appendix.

Notice that the only thing missing for the game to work with boards beyond 3*3 is to modify the board.__str__() function. Currently, that's the *only* thing hardcoded in the game for 3*3. If you use a bigger board as input, the game should work, except for __str__()

# 4. References
This section should detail any references you used when formulating your problem or producing your solution.

Heineman, G. T., Pollice, G., & Selkow, S. (2016). Algorithms in a nutshell: A practical guide. " O'Reilly Media, Inc.".

# 5. Appendices
Include here any relevant appendices (e.g. Python or Prolog code). Also include a copy of your original proposal here.

## Appendix A: Code

In [ ]:
#importing modules
import random
import math

In [289]:
class MathOps:
    """
        class MathOps: Provides helper functions for a tic tac toe game state representationa and calculation. 
        
        Notes:
        The goal of this class is to enable the tic tac toe game to scale for any n*n grid size
    """
    def __init__(self):
        """
        Initializing the class
        """
        pass

    def Transpose(self, matrix):
        """
        Returns the transpose of any n*n matrix. This function is adapted from a function I wrote for CS110 final at minerva (Badra, 2021)
        """
        cell = 0
        transposed_matrix = []
        for index in range(len(matrix[0])):
            temp_row = []
            for column in matrix:
                temp_row.append(column[cell])
            cell += 1
            transposed_matrix.append(temp_row)
        return transposed_matrix

    def rows(self, matrix):
        extracted_rows = []
        for row in matrix:
            extracted_rows.append(row)
        return extracted_rows

    def columns(self, matrix):
        transposed_matrix = self.Transpose(matrix)
        extracted_columns = []
        for column in transposed_matrix:
            extracted_columns.append(column)
        return extracted_columns

    def diagonals(self, matrix):
        """
        Returns the principal and minor diagonals of any n*n matrix
        """
        size = len(matrix)
        extracted_diagonals = []
        
        #extracting principle diagonal
        extracted_principle_diagonal = []
        for row_index, row in enumerate(matrix):
            for col_index, element in enumerate(row):
                if row_index == col_index:
                    extracted_principle_diagonal.append(matrix[row_index][col_index])
        extracted_diagonals.append(extracted_principle_diagonal)

        #extracting minor diagonal
        extracted_minor_diagonal = []
        for row_index, row in enumerate(matrix):
            for col_index, element in enumerate(row):
                if row_index + col_index == size - 1:
                    extracted_minor_diagonal.append(matrix[row_index][col_index])
        extracted_diagonals.append(list(reversed(extracted_minor_diagonal)))
            
        return(extracted_diagonals)

    def consecutives(self, matrix):
        """
        Returns a list of all consecutive elements in an n*n matrix, horizontally, vertically, and diagonally. Essentially, it returns all
        rows, columns, and diagonals.
        """
        return self.rows(matrix) + self.columns(matrix) + self.diagonals(matrix)

In [285]:
class Board(MathOps):
    """
    class Board: Represents a tic-tac-toe game node
    """
    def __init__(self, board_state):
        #inheriting parent classes' methods and attributes
        super().__init__()
        self.board_state = board_state
        #allowed player statuses at any state
        self.legal_residents = ["X", "O"]
        self.dimension = len(board_state)

    def __str__(self):
        """
        Returns tic-tac-toe board string representation. This function is adapted from my code for Assignment 2 (Badra, 2022)
        """
        print_string = ""

        print_string += "  _" + (self.dimension - 1) * "   _" + "\n"
        for row in self.board_state:
            print_string += "| "
            for col_index, cell_value in enumerate(row):
                if cell_value == 0:
                    cell_value = "*"
                print_string += (cell_value + " | ")
            print_string += "\n"
            print_string += (("  _" + (self.dimension - 1) * "   _") + "\n")
        return print_string

    def empty_cells(self):
        """
        Returns a list of indices of empty cells in board_state
        """
        empty_cells_indices = []
        for row_index, row in enumerate(self.board_state):
            for col_index, element in enumerate(row):
                if element == 0:
                    empty_cells_indices.append((row_index, col_index))

        return empty_cells_indices

    def is_win_state(self, player_symbol):
        """
        Returns whether a given player ("X" or "O") is at a win state or not
        """
        if player_symbol not in self.legal_residents:
            print("A player can only be 'X' or 'O'")
            return "Invalid input error"

        general_player_win_state = [player_symbol, player_symbol, player_symbol]

        win_states = self.consecutives(self.board_state)
        
        if general_player_win_state in win_states:
            return True
        else:
            return False

In [286]:
class TicTacToe(Board):
    """
        Class TicTacToe: implements a user facing n*n tic tac toe game with AI opponent
    """
    default_board_state = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]
    def __init__(self, board_state = default_board_state):
        self.current_game = Board(board_state)
        self.AI = 1
        self.USER = -1
        self.size = len(board_state)
        
        #As the tic tac toe board is initialized only once, I set the user symbol here in advance
        self.USER_SYMBOL, self.AI_SYMBOL = random.sample(self.current_game.legal_residents, k = 2)
        #self.USER_SYMBOL, self.AI_SYMBOL = "O", "X"

    def game_over(self, board_node):
        return board_node.is_win_state(self.USER_SYMBOL) or board_node.is_win_state(self.AI_SYMBOL)

    def input_to_tuple(self, user_input):
        """
        Returns all number in the user input
        """
        parsed_input = []
        for character in user_input:
            if character.isdigit():
                parsed_input.append(int(character))
        return tuple(parsed_input)

    def apply_move(self, player, move_cell_index):
        """
        Takes in a player and move_cell_index on the board and applies it if valid.

        Input:
            -player: self.USER or self.AI
            -move_cell_index: (TUPLE)
        """
        avaialable_cells = self.current_game.empty_cells()

        move_cell_row_index, move_cell_col_index = move_cell_index[0], move_cell_index[1]

        #print("HERE!")
        if tuple(move_cell_index) in self.current_game.empty_cells():
            if player == self.USER:
                #print("USER")
                self.current_game.board_state[move_cell_row_index][move_cell_col_index] = self.USER_SYMBOL
            elif player == self.AI:
                #print("AI")
                self.current_game.board_state[move_cell_row_index][move_cell_col_index] = self.AI_SYMBOL
        else:
            print("Invalid move!")
            print("Available moves", avaialable_cells)  
        #print(self.current_game.board_state)
    
    def user_turn(self):
        print("User Turn!")
        #print(self.current_game)
        avaialable_cells = self.current_game.empty_cells()

        user_play = None
        #the first condition is for checking whether the user played or not
        while not (user_play and (user_play in avaialable_cells)):
            user_input = self.input_to_tuple(input("Where do you want to play? (row, index)"))
            #print(len(user_input), user_input, type(user_input))

            if len(user_input) != 2:
                print("Please input the cell index in order. Ex: (Row_index, Col_index)")
            elif user_input not in avaialable_cells:
                print("Invalid move!")
                print("Available moves", avaialable_cells)
            else:
                user_play = user_input
                print("User move:", user_play)
                self.apply_move(self.USER, user_play)
                #print(self.current_game)

    def game_status(self, board_status):
        """
        Returns the following after evaluating the current state:
            -(positive 1): if the AI wins the current state
            -(negative 1): if the User wins the current state
            -(zero): both players draw
        """
        dummy_board_node = Board(board_status)
        if dummy_board_node.is_win_state(self.USER_SYMBOL):
            return -1
        elif dummy_board_node.is_win_state(self.AI_SYMBOL):
            return 1
        else:
            return 0        
    
    def minimax_optimal(self, board_state, layer, player):
        """
        Returns a list with [optimal_row, optimal_col, optimal_score]

        Input:
            - board_state: (n*n lists) board representation 
            - layer: (board node) level in the tic tac toe game tree
            - player: (int)
                -(positive 1): if the player is AI
                -(negative 1): if the player is USER
        """
        #set once and doesn't update
        playground_board = Board(board_state)
        #continously updates
        playground_board_state = playground_board.board_state
        avaialable_cells = self.current_game.empty_cells()
        
        if player == self.AI:
            best = [-1, -1, -math.inf]
        else:
            best = [-1, -1, +math.inf]

        if self.game_over(self.current_game) or layer == 0:
            game_score =  self.game_status(playground_board_state)
            return [-1, -1, game_score]
        
        for possible_cell_move in avaialable_cells:
            row_index, column_index = possible_cell_move[0], possible_cell_move[1]

            playground_board_state[row_index][column_index] = player
            game_score = self.minimax_optimal(playground_board_state, layer - 1, -player)
            playground_board_state[row_index][column_index] = 0
            game_score[0], game_score[1] = row_index, column_index

            if player == self.AI:
                if game_score[2] > best[2]:
                    #max case
                    best = game_score
            else:
                if game_score[2] < best[2]:
                    #min case
                    best = game_score
        return best

    def ai_turn(self):
        #print(self.current_game)
        print("AI Turn!")

        avaialable_cells = self.current_game.empty_cells()
        layer = len(avaialable_cells)
        if layer == 0 or self.game_over(self.current_game):
            return

        #if layer == self.size * self.size:
        #    optimal_move_row, optimal_move_col = random.sample(list(range(self.size)), k = 2)
        #else:
        #taking the first two indices: row_index and col_index
        optimal_move_row, optimal_move_col, best = self.minimax_optimal(self.current_game.board_state, layer, self.AI)
        move = [optimal_move_row, optimal_move_col]
        print("BEST:", best)

        print("AI move:", move)
        self.apply_move(self.AI, [move[0], move[1]])

    def execute(self):
        """
        Initializes the tic tac toe game
        """
        print("Welcome to tic-tac-toe n*n game!")
        current_mover = None

        while current_mover == None:
            user_response = input("Do you want to start first? y/n")
            if user_response.lower() in ["yes", "y"]:
                current_mover = self.USER
            elif user_response.lower() in ["no", "n"]:
                current_mover = self.AI = 1
            else:
                print("Please input either yes or no")
        print("You are {symbol}!".format(symbol = self.USER_SYMBOL))

        while not self.game_over(self.current_game):

            if current_mover == self.USER:
                print(self.current_game)
                self.user_turn()
                current_mover = self.AI

            elif current_mover == self.AI:
                print(self.current_game)
                self.ai_turn()
                current_mover = self.USER


        result = self.game_status(self.current_game)
        if result == 1:
            print("AI Wins!")
        elif result == -1:
            print("You Won!!!")
        else:
            print("It's a draw!")

In [293]:
#the AI takes the correct move: (0, 1)
z = TicTacToe([["X", 0, "X"], [0, 0, 0], [0, 0, 0]])
z.minimax_optimal(z.current_game.board_state, 2, 1)

[0, 1, 0]

In [292]:
#the AI takes the correct move: (2, 0)
z = TicTacToe([["X", "X", 0], [0, 0, 0], [0, 0, 0]])
z.minimax_optimal(z.current_game.board_state, 2, 1)

[0, 2, 0]

In [ ]:
#run the following function to play the game
state_zero = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]
z = TicTacToe(state_zero)
z.execute()

In [ ]:
"[[0, 0, 0], [0, 0, 0], [0, 0, 0]]"

## Appendix B: Original proposal

1. An AI player with minimax for a tic-tac-toe game
2. LOs: #aicoding, #search
3. I plan on using minimax algorithm with potentiall alpha beta pruning or q-learning to implement a tic-tac-toe AI algorithm. The deliverable will be a command line based tic tac toe game that a user can play against the AI.